In [31]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import tfb

pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

In [2]:
# general functions
COLOR_LIST = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

def load_pc_data(file, columns):
    """Loads PC data from file and puts it into a pandas dataframe
    """
    with open(file, 'r') as fin:
        pc_data = json.load(fin)

    pc_dict = {}
    for c in columns:
        d = []
        for pc in pc_data:
            d += [x[c] for x in pc_data[pc]]
        pc_dict[c] = d

    return pd.DataFrame(pc_dict)


def attack_hit_crit_prob(AC, AB):
    return max(0.05, min(0.95, 0.05*(21 + AB - AC)))

def save_fail_prob(SB, DC):
    return max(0.0, min(1.0, 0.05*(DC - SB - 1)))

def plot_data_and_fit(fig, x, y, **kwargs):
    print_coefficients = kwargs.pop('print_coefficients', False)
    # plot data
    kwargs['showlegend'] = False
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers', 
        **kwargs,
    ))

    # plot fit
    kwargs['hoverinfo'] = 'skip'
    kwargs['showlegend'] = True
    #if kwargs.get('line', None):
    #    kwargs['line']['dash'] = 'solid'

    coefs = np.polyfit(x, y, 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=x, y=poly(x),
        mode='lines', 
        **kwargs,
    ))
    if print_coefficients: 
        print(f'LVL >= 1: {coefs[1]:.2f} + {coefs[0]:.2f}*LVL')
    return fig

def average_save_bonus(row, **kwargs):
    saves = kwargs.get('saves', ['Str Save','Dex Save','Con Save','Int Save','Wis Save','Cha Save'])
    ave = np.mean([row[s] for s in saves])
    return ave

def count_save_proficiencies(row):
    return row['Str Prof'] + row['Dex Prof'] + row['Con Prof'] + row['Int Prof'] + row['Wis Prof'] + row['Cha Prof']

def unadjusted_armor_class(row):
    ac = row['adj AC']
    ac -= 2 if 'magic resistance' in row['AC adjustments'] else 0
    match count_save_proficiencies(row):
        case 3:
            ac -= 2
        case 4:
            ac -= 2
        case 5:
            ac -= 4
        case 6:
            ac -= 4
        case _:
            ac += 0
    return ac

In [3]:
# simple class for building player characters for defensive stats like AC and SB.
import numpy as np

def proficiency_bonus(lvl):
    return int(np.floor((lvl - 1)/4) + 2)

def ability_modifier(score):
    return int(np.floor(score/2) - 5)

class FeatureEquation:
    def __init__(self, name, equation):
        self.name = name
        self.set_equation(equation)

    def __repr__(self):
        return f'{{"name": "{self.name}", "equation": "{self.equation}"}}'
    
    def get_value(self, **kwargs):
        #return eval_expr(self.equation.format(**kwargs))
        return eval(self.equation.format(**kwargs))

    def get_equation(self):
        return self.equation
    
    def set_equation(self, equation):
        self.equation = equation
        return self

    def append_equation(self, equation):
        self.equation += equation
        return self


class Ability:
    def __init__(self, name, score_equation, save_equation):
        self.name = name
        self.set_score_equation(score_equation)
        self.set_save_equation(save_equation)

    def __repr__(self):
        return f'{{"name": "{self.name}", "score_equation": "{self.score_equation}", "save_equation": "{self.save_equation}"}}'
    
    def get_save_equation(self):
        return self.save_equation
    
    def get_score_equation(self):
        return self.score_equation
    
    def get_score(self):
        return eval(self.get_score_equation())
    
    def get_modifier(self):
        return ability_modifier(self.get_score())
    
    def get_save_bonus(self, **kwargs):
        #return eval_expr(self.save_equation.format(**kwargs))
        return eval(self.save_equation.format(**kwargs))
    
    def set_save_equation(self, equation):
        self.save_equation = equation
        return self
    
    def set_score_equation(self, equation):
        self.score_equation = equation
        return self

    def append_save_equation(self, equation):
        self.save_equation += equation
        return self
    
    def append_score_equation(self, equation):
        self.score_equation += equation
        return self


class PlayerCharacter:
    level = 0

    # ability scores
    str = Ability('Strength',     '8', '{STR_MOD}')
    dex = Ability('Dexterity',    '8', '{DEX_MOD}')
    con = Ability('Constitution', '8', '{CON_MOD}')
    int = Ability('Intelligence', '8', '{INT_MOD}')
    wis = Ability('Wisdom',       '8', '{WIS_MOD}')
    cha = Ability('Charisma',     '8', '{CHA_MOD}')

    # other stats
    armor_class = FeatureEquation('Armor Class', '10+{DEX_MOD}')
    attack_bonus = FeatureEquation('Attack Bonus', '{STR_MOD}+{PB}')
    save_dc = FeatureEquation('Attack Bonus', '8+{STR_MOD}+{PB}')

    # features
    features = []

    # methods
    def __init__(self, **kwargs):
        self.name = kwargs.get('name', None)
        self.level = kwargs.get('level', 0)
        self.features = kwargs.get('features', [])
    
    def apply_feature(self, feature):
        for n, d in feature.items():
            item = getattr(self, n)
            for m, v in d.items():
                method = getattr(item, m)
                result = method(v)

    def build(self):
        for feat in self.features:
            if feat['level'] <= self.level:
                self.apply_feature(feat['apply'])
        return self
    
    def get_ability_modifiers(self):
        return {
            'Strength':     self.str.get_modifier(),
            'Dexterity':    self.dex.get_modifier(),
            'Constitution': self.con.get_modifier(),
            'Intelligence': self.int.get_modifier(),
            'Wisdom':       self.wis.get_modifier(),
            'Charisma':     self.cha.get_modifier(),
        }
    
    def get_ability_scores(self):
        return {
            'Strength':     self.str.get_score(),
            'Dexterity':    self.dex.get_score(),
            'Constitution': self.con.get_score(),
            'Intelligence': self.int.get_score(),
            'Wisdom':       self.wis.get_score(),
            'Charisma':     self.cha.get_score(),
        }
    
    def get_attack_bonus(self):
        tokens = self.get_tokens()
        return self.attack_bonus.get_value(**tokens)
    
    def get_armor_class(self):
        tokens = self.get_tokens()
        return self.armor_class.get_value(**tokens)

    def get_level(self):
        return self.level
    
    def get_proficiency_bonus(self):
        return proficiency_bonus(self.get_level())
    
    def get_save_bonus(self, ability):
        return self.get_save_bonuses()[ability]
    
    def get_save_bonuses(self):
        tokens = self.get_tokens()
        return {
            'Strength':     self.str.get_save_bonus(**tokens),
            'Dexterity':    self.dex.get_save_bonus(**tokens),
            'Constitution': self.con.get_save_bonus(**tokens),
            'Intelligence': self.int.get_save_bonus(**tokens),
            'Wisdom':       self.wis.get_save_bonus(**tokens),
            'Charisma':     self.cha.get_save_bonus(**tokens),
        }
    
    def get_save_dc(self):
        tokens = self.get_tokens()
        return self.save_dc.get_value(**tokens)
    
    def get_tokens(self):
        tokens = {
            'PB': self.get_proficiency_bonus(),
            'LVL': self.get_level(),
            'STR_MOD': self.str.get_modifier(),
            'DEX_MOD': self.dex.get_modifier(),
            'CON_MOD': self.con.get_modifier(),
            'INT_MOD': self.int.get_modifier(),
            'WIS_MOD': self.wis.get_modifier(),
            'CHA_MOD': self.cha.get_modifier(),
            'FIN_MOD': max(self.str.get_modifier(), self.dex.get_modifier()),
        }
        return tokens
    
    def set_level(self, level):
        self.level = level
        return self

In [4]:
# class builds for defensive stats figures
PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}
"""
Barbarian:  unarmored, +3 Con, +2 Dex; +4 Con @ lv 12; +5 Con @ lv 16; +7 Con @ lv 20;
Bard:       leather armor, +3 Dex; studded leather armor @ lv 5;
Cleric:     scale mail, shield, +2 Dex; half plate @ lv 7;
Druid:      leather armor, shield, +2 Dex; studded leather armor @ lv 5; scale mail @ lv 7;
Fighter:    chain mail; plate @ lv 10;
Monk:       unarmored, +3 Dex, +3 Wis; +4 Dex @ lv 4; +5 Dex @ lvl 8; +4 Wis @ lv 12; +5 Wis @ lvl 16; empty body @ lv 17;
Paladin:    chain mail; plate @ lv 10;
Ranger:     leather armor, +3 Dex; +4 Dex @ lv 4; +5 Dex @ lv 5;
Rogue:      leather armor, +3 Dex; +4 Dex @ lv 4; +5 Dex @ lv 5;
Sorcerer:   none, +2 Dex; mage armor @ lv 5; shield spell @ lv 11;
Warlock:    leather armor, +2 Dex; mage armor @ lv 5;
Wizard:     none, +2 Dex; mage armor @ lv 5; shield spell @ lv 11;
"""

pcs = {}

pcs['Barbarian'] = PlayerCharacter(
    name='Barbarian',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation': '15+1', 'set_save_equation': '{STR_MOD}+{PB}'},
                'dex': {'set_score_equation': '13+1', 'set_save_equation': '{DEX_MOD}'},
                'con': {'set_score_equation': '15+1', 'set_save_equation': '{CON_MOD}+{PB}'},
                'int': {'set_score_equation':    '8', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation':   '12', 'set_save_equation': '{WIS_MOD}'},
                'cha': {'set_score_equation':    '8', 'set_save_equation': '{CHA_MOD}'},
                'armor_class': {'set_equation': '10+{DEX_MOD}+{CON_MOD}'},
                'attack_bonus': {'set_equation': '{STR_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{STR_MOD}+{PB}'},
            }
        ),
        dict(level=2,  name='Danger Sense', apply={'dex': {'append_save_equation': '+4'}}),
        dict(level=4,  name='ASI', apply={'str': {'append_score_equation': '+2'}}),
        dict(level=8,  name='ASI', apply={'str': {'append_score_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=19, name='Primal Champion', apply={'str': {'append_score_equation': '+4'}, 'con': {'append_score_equation': '+4'}}),
    ]
)

pcs['Bard'] = PlayerCharacter(
    name='Bard',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation':    '8', 'set_save_equation': '{STR_MOD}'},
                'dex': {'set_score_equation': '14+2', 'set_save_equation': '{DEX_MOD}+{PB}'},
                'con': {'set_score_equation':   '12', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation':   '13', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation':   '10', 'set_save_equation': '{WIS_MOD}'},
                'cha': {'set_score_equation': '15+1', 'set_save_equation': '{CHA_MOD}+{PB}'},
                'attack_bonus': {'set_equation': '{CHA_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{CHA_MOD}+{PB}'},
            }
        ),
        dict(level=1,  name='Leather Armor', apply={'armor_class': {'set_equation': '11+{DEX_MOD}'}}),
        dict(level=4,  name='ASI', apply={'cha': {'append_score_equation': '+2'}}),
        dict(level=5,  name='Studded Leather Armor', apply={'armor_class': {'set_equation': '12+{DEX_MOD}'}}),
        dict(level=8,  name='ASI', apply={'cha': {'append_score_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'int': {'append_score_equation': '+2'}}),
    ]
)

pcs['Cleric'] = PlayerCharacter(
    name='Cleric',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation': '13+1', 'set_save_equation': '{STR_MOD}'},
                'dex': {'set_score_equation': '13+1', 'set_save_equation': '{DEX_MOD}'},
                'con': {'set_score_equation':   '12', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation':   '10', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation': '15+1', 'set_save_equation': '{WIS_MOD}+{PB}'},
                'cha': {'set_score_equation':   '10', 'set_save_equation': '{CHA_MOD}+{PB}'},
                'attack_bonus': {'set_equation': '{WIS_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{WIS_MOD}+{PB}'},
            }
        ),
        dict(level=1,  name='Scale Mail', apply={'armor_class': {'set_equation': '14+min(2, {DEX_MOD})'}}),
        dict(level=1,  name='Shield', apply={'armor_class': {'append_equation': '+2'}}),
        dict(level=4,  name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=7,  name='Half Plate', apply={'armor_class': {'set_equation': '15+min(2, {DEX_MOD})'}}),
        dict(level=7,  name='Shield', apply={'armor_class': {'append_equation': '+2'}}),
        dict(level=8,  name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'str': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
    ]
)

pcs['Druid'] = PlayerCharacter(
    name='Druid',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation':    '8', 'set_save_equation': '{STR_MOD}'},
                'dex': {'set_score_equation': '13+1', 'set_save_equation': '{DEX_MOD}'},
                'con': {'set_score_equation': '13+1', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation':   '12', 'set_save_equation': '{INT_MOD}+{PB}'},
                'wis': {'set_score_equation': '15+1', 'set_save_equation': '{WIS_MOD}+{PB}'},
                'cha': {'set_score_equation':   '12', 'set_save_equation': '{CHA_MOD}'},
                'attack_bonus': {'set_equation': '{WIS_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{WIS_MOD}+{PB}'},
            }
        ),
        dict(level=1,  name='Leather Armor', apply={'armor_class': {'set_equation': '11+{DEX_MOD}'}}),
        dict(level=1,  name='Shield', apply={'armor_class': {'append_equation': '+2'}}),
        dict(level=4,  name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=5,  name='Studded Leather Armor', apply={'armor_class': {'set_equation': '12+{DEX_MOD}'}}),
        dict(level=5,  name='Shield', apply={'armor_class': {'append_equation': '+2'}}),
        dict(level=8,  name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
    ]
)

pcs['Fighter'] = PlayerCharacter(
    name='Fighter',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation': '14+2', 'set_save_equation': '{STR_MOD}+{PB}'},
                'dex': {'set_score_equation':   '10', 'set_save_equation': '{DEX_MOD}'},
                'con': {'set_score_equation': '15+1', 'set_save_equation': '{CON_MOD}+{PB}'},
                'int': {'set_score_equation':   '10', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation':   '12', 'set_save_equation': '{WIS_MOD}'},
                'cha': {'set_score_equation':   '11', 'set_save_equation': '{CHA_MOD}'},
                'attack_bonus': {'set_equation': '{STR_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{STR_MOD}+{PB}'},
            }
        ),
        dict(level=1,  name='Chainmail', apply={'armor_class': {'set_equation': '16'}}),
        dict(level=4,  name='ASI', apply={'str': {'append_score_equation': '+2'}}),
        dict(level=6,  name='ASI', apply={'str': {'append_score_equation': '+2'}}),
        dict(level=8,  name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=9,  name='Indomitable', apply={
            'str': {'append_save_equation': '+0.33'},
            'dex': {'append_save_equation': '+0.33'},
            'con': {'append_save_equation': '+0.33'},
            'int': {'append_save_equation': '+0.33'},
            'wis': {'append_save_equation': '+0.33'},
            'cha': {'append_save_equation': '+0.33'},
        }),
        dict(level=10, name='Plate', apply={'armor_class': {'set_equation': '18'}}),
        dict(level=12, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=13, name='Indomitable', apply={
            'str': {'append_save_equation': '+0.33'},
            'dex': {'append_save_equation': '+0.33'},
            'con': {'append_save_equation': '+0.33'},
            'int': {'append_save_equation': '+0.33'},
            'wis': {'append_save_equation': '+0.33'},
            'cha': {'append_save_equation': '+0.33'},
        }),
        dict(level=14, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=17, name='Indomitable', apply={
            'str': {'append_save_equation': '+0.33'},
            'dex': {'append_save_equation': '+0.33'},
            'con': {'append_save_equation': '+0.33'},
            'int': {'append_save_equation': '+0.33'},
            'wis': {'append_save_equation': '+0.33'},
            'cha': {'append_save_equation': '+0.33'},
        }),
        dict(level=19, name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
    ]
)

pcs['Monk'] = PlayerCharacter(
    name='Monk',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation':   '12', 'set_save_equation': '{STR_MOD}+{PB}'},
                'dex': {'set_score_equation': '15+1', 'set_save_equation': '{DEX_MOD}+{PB}'},
                'con': {'set_score_equation': '13+1', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation':    '8', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation': '15+1', 'set_save_equation': '{WIS_MOD}'},
                'cha': {'set_score_equation':    '8', 'set_save_equation': '{CHA_MOD}'},
                'armor_class': {'set_equation': '10+{DEX_MOD}+{WIS_MOD}'},
                'attack_bonus': {'set_equation': '{DEX_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{WIS_MOD}+{PB}'},
            }
        ),
        dict(level=4,  name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=7,  name='Evasion', apply={'dex': {'append_save_equation': '+6'}}), # mathematically closer to +8, assuming normal chance to fail
        dict(level=8,  name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=14, name='Diamond Soul', apply={
            'con': {'append_save_equation': '+{PB}'},
            'int': {'append_save_equation': '+{PB}'},
            'wis': {'append_save_equation': '+{PB}'},
            'cha': {'append_save_equation': '+{PB}'},
        }),
        dict(level=16, name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
    ]
)

pcs['Paladin'] = PlayerCharacter(
    name='Paladin',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation': '15+1', 'set_save_equation': '{STR_MOD}'},
                'dex': {'set_score_equation':   '10', 'set_save_equation': '{DEX_MOD}'},
                'con': {'set_score_equation': '13+1', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation':   '10', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation':    '8', 'set_save_equation': '{WIS_MOD}+{PB}'},
                'cha': {'set_score_equation': '15+1', 'set_save_equation': '{CHA_MOD}+{PB}'},
                'attack_bonus': {'set_equation': '{STR_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{CHA_MOD}+{PB}'},
            }
        ),
        dict(level=1,  name='Chainmail', apply={'armor_class': {'set_equation': '16'}}),
        dict(level=4, name='ASI', apply={'str': {'append_score_equation': '+2'}}),
        dict(
            level=6, 
            name='Aura of Protection', 
            apply={
                'str': {'append_save_equation': '+{CHA_MOD}'},
                'dex': {'append_save_equation': '+{CHA_MOD}'},
                'con': {'append_save_equation': '+{CHA_MOD}'},
                'int': {'append_save_equation': '+{CHA_MOD}'},
                'wis': {'append_save_equation': '+{CHA_MOD}'},
                'cha': {'append_save_equation': '+{CHA_MOD}'},
            }
        ),
        dict(level=8,  name='ASI', apply={'str': {'append_score_equation': '+2'}}),
        dict(level=10, name='Plate', apply={'armor_class': {'set_equation': '18'}}),
        dict(level=12, name='ASI', apply={'cha': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'cha': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
    ]
)

pcs['Ranger'] = PlayerCharacter(
    name='Ranger',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation':   '12', 'set_save_equation': '{STR_MOD}+{PB}'},
                'dex': {'set_score_equation': '14+2', 'set_save_equation': '{DEX_MOD}+{PB}'},
                'con': {'set_score_equation':   '12', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation':   '10', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation': '15+1', 'set_save_equation': '{WIS_MOD}'},
                'cha': {'set_score_equation':    '9', 'set_save_equation': '{CHA_MOD}'},
                'attack_bonus': {'set_equation': '{DEX_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{WIS_MOD}+{PB}'},
            }
        ),
        dict(level=1,  name='Leather Armor', apply={'armor_class': {'set_equation': '11+{DEX_MOD}'}}),
        dict(level=4,  name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=5,  name='Studded Leather Armor', apply={'armor_class': {'set_equation': '12+{DEX_MOD}'}}),
        dict(level=8,  name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
    ]
)

pcs['Rogue'] = PlayerCharacter(
    name='Rogue',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation':    '8', 'set_save_equation': '{STR_MOD}'},
                'dex': {'set_score_equation': '14+2', 'set_save_equation': '{DEX_MOD}+{PB}'},
                'con': {'set_score_equation': '13+1', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation':   '14', 'set_save_equation': '{INT_MOD}+{PB}'},
                'wis': {'set_score_equation':   '12', 'set_save_equation': '{WIS_MOD}'},
                'cha': {'set_score_equation':   '12', 'set_save_equation': '{CHA_MOD}'},
                'attack_bonus': {'set_equation': '{DEX_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{DEX_MOD}+{PB}'},
            }
        ),
        dict(level=1,  name='Leather Armor', apply={'armor_class': {'set_equation': '11+{DEX_MOD}'}}),
        dict(level=4,  name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=5,  name='Studded Leather Armor', apply={'armor_class': {'set_equation': '12+{DEX_MOD}'}}),
        dict(level=7,  name='Evasion', apply={'dex': {'append_save_equation': '+6'}}), # mathematically closer to +8, assuming normal chance to fail
        dict(level=8,  name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=15, name='Slippery Mind', apply={'wis': {'append_save_equation': '+{PB}'}}),
        dict(level=16, name='ASI', apply={'wis': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
    ]
)

pcs['Sorcerer'] = PlayerCharacter(
    name='Sorcerer',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation':    '8', 'set_save_equation': '{STR_MOD}'},
                'dex': {'set_score_equation':   '14', 'set_save_equation': '{DEX_MOD}'},
                'con': {'set_score_equation': '13+1', 'set_save_equation': '{CON_MOD}+{PB}'},
                'int': {'set_score_equation':   '12', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation':   '12', 'set_save_equation': '{WIS_MOD}'},
                'cha': {'set_score_equation': '14+2', 'set_save_equation': '{CHA_MOD}+{PB}'},
                'attack_bonus': {'set_equation': '{CHA_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{CHA_MOD}+{PB}'},
            }
        ),
        dict(level=4,  name='ASI', apply={'cha': {'append_score_equation': '+2'}}),
        dict(level=5,  name='Mage Armor', apply={'armor_class': {'set_equation': '13+{DEX_MOD}'}}),
        dict(level=8,  name='ASI', apply={'cha': {'append_score_equation': '+2'}}),
        dict(level=11, name='Shield', apply={'armor_class': {'append_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
    ]
)

pcs['Warlock'] = PlayerCharacter(
    name='Warlock',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation':    '8', 'set_save_equation': '{STR_MOD}'},
                'dex': {'set_score_equation':   '14', 'set_save_equation': '{DEX_MOD}'},
                'con': {'set_score_equation': '13+1', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation':   '12', 'set_save_equation': '{INT_MOD}'},
                'wis': {'set_score_equation':   '12', 'set_save_equation': '{WIS_MOD}+{PB}'},
                'cha': {'set_score_equation': '14+2', 'set_save_equation': '{CHA_MOD}+{PB}'},
                'attack_bonus': {'set_equation': '{CHA_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{CHA_MOD}+{PB}'},
            }
        ),
        dict(level=1,  name='Leather Armor', apply={'armor_class': {'set_equation': '11+{DEX_MOD}'}}),
        dict(level=4,  name='ASI', apply={'cha': {'append_score_equation': '+2'}}),
        dict(level=5,  name='Studded Leather Armor', apply={'armor_class': {'set_equation': '12+{DEX_MOD}'}}),
        dict(level=8,  name='ASI', apply={'cha': {'append_score_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
    ]
)

pcs['Wizard'] = PlayerCharacter(
    name='Wizard',
    level=1,
    features=[
        dict(
            level=1, 
            name='Base Class', 
            apply={
                'str': {'set_score_equation':    '8', 'set_save_equation': '{STR_MOD}'},
                'dex': {'set_score_equation':   '14', 'set_save_equation': '{DEX_MOD}'},
                'con': {'set_score_equation': '13+1', 'set_save_equation': '{CON_MOD}'},
                'int': {'set_score_equation': '14+2', 'set_save_equation': '{INT_MOD}+{PB}'},
                'wis': {'set_score_equation':   '12', 'set_save_equation': '{WIS_MOD}+{PB}'},
                'cha': {'set_score_equation':   '12', 'set_save_equation': '{CHA_MOD}'},
                'attack_bonus': {'set_equation': '{INT_MOD}+{PB}'},
                'save_dc': {'set_equation': '8+{INT_MOD}+{PB}'},
            }
        ),
        dict(level=4,  name='ASI', apply={'int': {'append_score_equation': '+2'}}),
        dict(level=5,  name='Mage Armor', apply={'armor_class': {'set_equation': '13+{DEX_MOD}'}}),
        dict(level=8,  name='ASI', apply={'int': {'append_score_equation': '+2'}}),
        dict(level=11, name='Shield', apply={'armor_class': {'append_equation': '+2'}}),
        dict(level=12, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=16, name='ASI', apply={'con': {'append_score_equation': '+2'}}),
        dict(level=19, name='ASI', apply={'dex': {'append_score_equation': '+2'}}),
    ]
)


In [98]:
# puts staring stats into a table

data = {
    'Class': [],
    'Strength': [],
    'Dexterity': [],
    'Constitution': [],
    'Intelligence': [],
    'Wisdom': [],
    'Charisma': [],
}
for c in pcs:
    stats = pcs[c].set_level(1).build().get_ability_scores()
    data['Class'].append(c)
    data['Strength'].append(stats['Strength'])
    data['Dexterity'].append(stats['Dexterity'])
    data['Constitution'].append(stats['Constitution'])
    data['Intelligence'].append(stats['Intelligence'])
    data['Wisdom'].append(stats['Wisdom'])
    data['Charisma'].append(stats['Charisma'])

dfPC = pd.DataFrame(data)
dfPC = dfPC.style.format({
    'Class': '{:}',
    'Strength': '{:,.0f}',
    'Dexterity': '{:,.0f}',
    'Constitution': '{:,.0f}',
    'Intelligence': '{:,.0f}',
    'Wisdom': '{:,.0f}',
    'Charisma': '{:,.0f}',
}).hide().set_table_styles([
        {'selector': 'th', 'props': 'text-align: center;'},
        {'selector': 'td', 'props': 'text-align: center;'},
    ], overwrite=False)
dfPC.to_html(
    './tab-pc-baseline-ability-scores.html',
    index=False, classes='wide', border=0)
dfPC

Class,Strength,Dexterity,Constitution,Intelligence,Wisdom,Charisma
Barbarian,16,14,16,8,12,8
Bard,8,16,12,13,10,16
Cleric,14,14,12,10,16,10
Druid,8,14,14,12,16,12
Fighter,16,10,16,10,12,11
Monk,12,16,14,8,16,8
Paladin,16,10,14,10,8,16
Ranger,12,16,12,10,16,9
Rogue,8,16,14,14,12,12
Sorcerer,8,14,14,12,12,16


In [173]:
# Plots attack bonus for each class.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='attack bonus',
            range=[4, 14],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.01,
            xanchor='left', x=0.00,
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# get PC data
levels = list(range(1,21))
for c in pcs:
    pc = pcs[c]
    ab = [pc.set_level(lvl).build().get_attack_bonus() for lvl in levels]
    fig.add_trace(go.Scatter(
        x=levels, 
        y=ab,
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[c]['color'], width=1),
        marker=dict(color=PC_CLASSES[c]['color']),
        name=c,
        hovertemplate=
                f'<b>{c}</b><br>'+
                'level %{x:.0f}<br>'+
                'attack bonus %{y:.0f}'+
                '<extra></extra>'
    ))

# get PC data
levels = list(range(1,21))
ab_matrix = np.zeros([20,len(PC_CLASSES)])
pc_names = list(pcs.keys())
for i in range(len(pc_names)):
    pc = pcs[pc_names[i]]
    ab_matrix[:,i] = np.array([pc.set_level(lvl).build().get_attack_bonus() for lvl in levels])

plot_data_and_fit(
    fig, 
    levels, np.mean(ab_matrix, axis=1), 
    name='average',
    legendgroup='average',
    line=dict(color='black', dash='dash'),
    hovertemplate=
            f'<b>Average</b><br>'+
            'level %{x:.0f}<br>'+
            'attack bonus %{y:.1f}'+
            '<extra></extra>',
    print_coefficients=True,
)

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-ab-vs-level-large', style='aspect-ratio:600/550')
    tfb.save_fig_html(fig, format='small', name=f'./fig-ab-vs-level-small', style='aspect-ratio:600/600')

LVL >= 1: 4.84 + 0.35*LVL


In [172]:
# Plots save difficulty class for each class.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save difficulty class',
            range=[12, 22],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.01,
            xanchor='left', x=0.00,
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# get PC data
exclude_pcs = ['Barbarian','Fighter','Rogue']
levels = list(range(1,21))
for c in pcs:
    if c in exclude_pcs: continue
    pc = pcs[c]
    dc = [pc.set_level(lvl).build().get_save_dc() for lvl in levels]
    fig.add_trace(go.Scatter(
        x=levels, 
        y=dc,
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[c]['color'], width=1),
        marker=dict(color=PC_CLASSES[c]['color']),
        name=c,
        hovertemplate=
                f'<b>{c}</b><br>'+
                'level %{x:.0f}<br>'+
                'difficulty class %{y:.0f}'+
                '<extra></extra>'
    ))

# get PC data
levels = list(range(1,21))
dc_matrix = np.zeros([20,len(PC_CLASSES)])
pc_names = list(pcs.keys())
for i in range(len(pc_names)):
    if c in exclude_pcs: continue
    pc = pcs[pc_names[i]]
    dc_matrix[:,i] = np.array([pc.set_level(lvl).build().get_save_dc() for lvl in levels])

plot_data_and_fit(
    fig, 
    levels, np.mean(dc_matrix, axis=1), 
    name='average',
    legendgroup='average',
    line=dict(color='black', dash='dash'),
    hovertemplate=
            f'<b>Average</b><br>'+
            'level %{x:.0f}<br>'+
            'difficulty class %{y:.1f}'+
            '<extra></extra>',
    print_coefficients=True,
)

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-dc-vs-level-large', style='aspect-ratio:600/550')
    tfb.save_fig_html(fig, format='small', name=f'./fig-dc-vs-level-small', style='aspect-ratio:600/550')

LVL >= 1: 12.58 + 0.36*LVL


In [32]:
# Plots probability of a PC hitting a monster with an attack or a monster failing a saving throw.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# monster stats
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['unadj AC'] = dfM.apply(lambda row: unadjusted_armor_class(row), axis=1)
dfM['adj SB'] = dfM.apply(lambda row: average_save_bonus(row), axis=1)
dfM['adj SB'] = dfM['adj SB'] + dfM.apply(lambda row: 4 if 'magic resistance' in row['AC adjustments'] else 0, axis=1)
dfM['adj DC'] = dfM['adj AB'] + 8
dfM = dfM[['CR','adj HP','AC','adj AC','adj SB','adj DPR','adj AB','adj DC','unadj AC']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 20)]
dfM.sort_values(by='CR', inplace=True)
m_ac  = dfM['unadj AC'].to_numpy()
m_sb  = dfM['adj SB'].to_numpy()

# player character data
levels = list(range(1,21))
ab_matrix = []
dc_matrix = []
exclude_pcs = ['Barbarian','Fighter','Rogue']
for c in pcs:
    pc = pcs[c]
    ab_matrix.append([pc.set_level(lvl).build().get_attack_bonus() for lvl in levels])
    if c not in exclude_pcs:
        dc_matrix.append([pc.set_level(lvl).build().get_save_dc() for lvl in levels])
p_ab = np.mean(np.array(ab_matrix), axis=0)
p_dc = np.mean(np.array(dc_matrix), axis=0)

# attacks
p_hit = np.array([attack_hit_crit_prob(ac, ab) for ac, ab in zip(m_ac, p_ab)])
plot_data_and_fit(
    fig, 
    levels, p_hit,
    name='attacks',
    legendgroup='attacks',
    line=dict(color=COLOR_LIST[0], dash='solid', width=1),
    hovertemplate=
            f'<b>Attacks</b><br>'+
            'level %{x:.0f}<br>'+
            'probability %{y:.1%}'+
            '<extra></extra>',
    print_coefficients=True,
)
print(p_hit.mean())


# saving throws
p_fail = np.array([save_fail_prob(sb, dc) for sb, dc in zip(m_sb, p_dc)])
plot_data_and_fit(
    fig, 
    levels, p_fail,
    name='saving throws',
    legendgroup='saving throws',
    line=dict(color=COLOR_LIST[1], dash='solid', width=1),
    hovertemplate=
            f'<b>Saving Throws</b><br>'+
            'level %{x:.0f}<br>'+
            'probability %{y:.1%}'+
            '<extra></extra>',
    print_coefficients=True,
)
print(p_fail.mean())

fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[0.65,0.65],
    mode='lines', 
    name='baseline',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip',
    showlegend=False,
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-hit-fail-probabilities-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-hit-fail-probabilities-small')

LVL >= 1: 0.58 + 0.00*LVL
0.6316528884426162
LVL >= 1: 0.57 + -0.00*LVL
0.5231603791025918


In [171]:
# Plots DPR for each class, calculated for a full adventuring day with Medium encounters and two short rests.
import pandas as pd

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='damage per round',
            range=[0, 70],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            yanchor='bottom',  y=1.01,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# get PC data
classes = list(PC_CLASSES.keys())
encounter_difficulty = 'Medium'
pc_data_file = f'../../_classes/xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json'
df = load_pc_data(pc_data_file, ['class','level','damage per round mean'])
df = df[df['class'].isin(classes)]

for c in classes:
    dfG = df[df['class'].eq(c)][['level','damage per round mean']].groupby(['level']).median().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['level'].to_numpy(), 
        y=dfG['damage per round mean'].to_numpy(),
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[c]['color'], width=1),
        marker=dict(color=PC_CLASSES[c]['color']),
        unselected=dict(marker=dict(opacity=0.0)),
        name=c,
        hovertemplate=
                f'<b>{c}</b><br>'+
                'level %{x}<br>'+
                'damage per round %{y:.1f}'+
                '<extra></extra>'
    ))


dfG = df[['level','damage per round mean']].groupby(['level']).mean().reset_index()
plot_data_and_fit(
    fig, 
    dfG['level'].to_numpy(), dfG['damage per round mean'].to_numpy(), 
    name='average',
    legendgroup='average',
    line=dict(color='black', dash='dash'),
    hovertemplate=
            '<b>Average</b><br>'+
            'level %{x:.0f}<br>'+
            'damage per round %{y:.1f}'+
            '<extra></extra>',
    print_coefficients=True,
)

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-dpr-vs-level-large', style='aspect-ratio:600/550')
    tfb.save_fig_html(fig, format='small', name=f'./fig-dpr-vs-level-small', style='aspect-ratio:600/600')

LVL >= 1: 7.09 + 2.20*LVL


In [170]:
# Plots armor class for each class.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='armor class',
            range=[12, 22],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.01,
            xanchor='left', x=0.00,
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# get PC data
levels = list(range(1,21))
for c in pcs:
    pc = pcs[c]
    ac = [pc.set_level(lvl).build().get_armor_class() for lvl in levels]
    fig.add_trace(go.Scatter(
        x=levels, 
        y=ac,
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[c]['color'], width=1),
        marker=dict(color=PC_CLASSES[c]['color']),
        name=c,
        hovertemplate=
                f'<b>{c}</b><br>'+
                'level %{x:.0f}<br>'+
                'armor class %{y:.0f}'+
                '<extra></extra>'
    ))

# get PC data
levels = list(range(1,21))
ac_matrix = np.zeros([20,len(PC_CLASSES)])
pc_names = list(pcs.keys())
for i in range(len(pc_names)):
    pc = pcs[pc_names[i]]
    ac_matrix[:,i] = np.array([pc.set_level(lvl).build().get_armor_class() for lvl in levels])

plot_data_and_fit(
    fig, 
    levels, np.mean(ac_matrix, axis=1), 
    name='average',
    legendgroup='average',
    line=dict(color='black', dash='dash'),
    hovertemplate=
            f'<b>Average</b><br>'+
            'level %{x:.0f}<br>'+
            'armor class %{y:.1f}'+
            '<extra></extra>',
    print_coefficients=True,
)

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-ac-vs-level-large', style='aspect-ratio:600/550')
    tfb.save_fig_html(fig, format='small', name=f'./fig-ac-vs-level-small', style='aspect-ratio:600/600')

LVL >= 1: 14.67 + 0.17*LVL


In [169]:
# Plots average saving throw bonus for each class, calculated for a full adventuring day with Medium encounters and two short rests.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='average save bonus',
            range=[0, 10],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.01,
            xanchor='left', x=0.00,
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# get PC data
levels = list(range(1,21))
for c in pcs:
    pc = pcs[c]
    sb_ave = [np.mean(list(pc.set_level(lvl).build().get_save_bonuses().values())) for lvl in levels]
    fig.add_trace(go.Scatter(
        x=levels, 
        y=sb_ave,
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[c]['color'], width=1),
        marker=dict(color=PC_CLASSES[c]['color']),
        name=c,
        hovertemplate=
                f'<b>{c}</b><br>'+
                'level %{x:.0f}<br>'+
                'save bonus %{y:.1f}'+
                '<extra></extra>'
    ))

# get PC data
levels = list(range(1,21))
sb_matrix = np.zeros([20,len(PC_CLASSES)])
pc_names = list(pcs.keys())
for i in range(len(pc_names)):
    pc = pcs[pc_names[i]]
    sb_matrix[:,i] = np.array([np.mean(list(pc.set_level(lvl).build().get_save_bonuses().values())) for lvl in levels])

plot_data_and_fit(
    fig, 
    levels, np.mean(sb_matrix, axis=1), 
    name='average',
    legendgroup='average',
    line=dict(color='black', dash='dash'),
    hovertemplate=
            f'<b>Average</b><br>'+
            'level %{x:.0f}<br>'+
            'save bonus %{y:.1f}'+
            '<extra></extra>',
    print_coefficients=True,
)

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sb-vs-level-large', style='aspect-ratio:600/550')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sb-vs-level-small', style='aspect-ratio:600/600')

LVL >= 1: 1.51 + 0.20*LVL


In [33]:
# Plots probability of a PC failing a saving throw against a level appropriate monster.

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[0, 1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.00,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# player character data
levels = list(range(1,21))
ac_matrix = []
sb_matrix = []
pc_names = list(pcs.keys())
exclude_pcs = ['Barbarian','Fighter','Rogue']
for c in pcs:
    pc = pcs[c]
    ac_matrix.append([pc.set_level(lvl).build().get_armor_class() for lvl in levels])
    sb_matrix.append([np.mean(list(pc.set_level(lvl).build().get_save_bonuses().values())) for lvl in levels])
p_ac = np.mean(np.array(ac_matrix), axis=0)
p_sb = np.mean(np.array(sb_matrix), axis=0)

# monster data
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['adj DC'] = dfM['adj AB'] + 8
dfM = dfM[['CR','adj AB','adj DC']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 20)]
dfM.sort_values(by='CR', inplace=True)
m_ab  = dfM['adj AB'].to_numpy()
m_dc  = dfM['adj DC'].to_numpy()

# attack
levels = list(range(1,21))
hit_prob = [attack_hit_crit_prob(ac, ab) for ac, ab in zip(p_ac, m_ab)]
plot_data_and_fit(
    fig, 
    levels, hit_prob,
    name='attacks',
    legendgroup='attacks',
    line=dict(color=COLOR_LIST[0], dash='solid', width=1),
    hovertemplate=
            f'<b>Attacks</b><br>'+
            'level %{x:.0f}<br>'+
            'probability %{y:.1%}'+
            '<extra></extra>',
    print_coefficients=True,
)

# saving throw
levels = list(range(1,21))
fail_prob = [save_fail_prob(sb, dc) for sb, dc in zip(p_sb, m_dc)]
plot_data_and_fit(
    fig, 
    levels, fail_prob,
    name='saving throws',
    legendgroup='saving throws',
    line=dict(color=COLOR_LIST[1], dash='solid', width=1),
    hovertemplate=
            f'<b>Saving Throws</b><br>'+
            'level %{x:.0f}<br>'+
            'probability %{y:.1%}'+
            '<extra></extra>',
    print_coefficients=True,
)

fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[0.65,0.65],
    mode='lines', 
    name='baseline',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip',
    showlegend=False,
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-defense-probabilities-vs-level-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-defense-probabilities-vs-level-small')

LVL >= 1: 0.53 + 0.01*LVL
LVL >= 1: 0.49 + 0.01*LVL


In [168]:
# Plots adjusted AC for each class.
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='adjusted armor class',
            range=[12, 22],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            yanchor='bottom',  y=1.01,
            xanchor='left', x=0.00,
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# get PC data
levels = list(range(1,21))
for c in pcs:
    pc = pcs[c]
    sb_ave = np.array([np.mean(list(pc.set_level(lvl).build().get_save_bonuses().values())) for lvl in levels])
    ac_ave = np.array([pc.set_level(lvl).build().get_armor_class() for lvl in levels])
    ef_ave = (sb_ave + 14 + ac_ave)/2
    fig.add_trace(go.Scatter(
        x=levels, 
        y=ef_ave,
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[c]['color'], width=1),
        marker=dict(color=PC_CLASSES[c]['color']),
        name=c,
        hovertemplate=
                f'<b>{c}</b><br>'+
                'level %{x:.0f}<br>'+
                'adj armor class %{y:.1f}'+
                '<extra></extra>'
    ))

# get PC data
levels = list(range(1,21))
sb_matrix = np.zeros([20,len(PC_CLASSES)])
ac_matrix = np.zeros([20,len(PC_CLASSES)])
ef_matrix = np.zeros([20,len(PC_CLASSES)])
pc_names = list(pcs.keys())
for i in range(len(pc_names)):
    pc = pcs[pc_names[i]]
    sb_matrix[:,i] = np.array([np.mean(list(pc.set_level(lvl).build().get_save_bonuses().values())) for lvl in levels])
    ac_matrix[:,i] = np.array([pc.set_level(lvl).build().get_armor_class() for lvl in levels])
    ef_matrix[:,i] = (sb_matrix[:,i] + 14 + ac_matrix[:,i])/2

plot_data_and_fit(
    fig, 
    levels, np.mean(ef_matrix, axis=1), 
    name='average',
    legendgroup='average',
    line=dict(color='black', dash='dash'),
    hovertemplate=
            f'<b>Average</b><br>'+
            'level %{x:.0f}<br>'+
            'save bonus %{y:.1f}'+
            '<extra></extra>',
    print_coefficients=True,
)

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-sb-vs-level-large', style='aspect-ratio:600/550')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-sb-vs-level-small', style='aspect-ratio:600/600')

LVL >= 1: 15.27 + 0.17*LVL


In [176]:
# Plots adjusted hit points for each class, calculated for a full adventuring day with Medium encounters and two short rests.
import pandas as pd

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='hit points',
            range=[0, 500],
            tick0=0, dtick=100,
            minor=dict(tick0=0, dtick=50),
        ),
        legend=dict(
            yanchor='bottom', y=1.01,
            xanchor='left',   x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# get PC data
classes = list(PC_CLASSES.keys())
encounter_difficulty = 'Medium'
pc_data_file = f'../../_classes/xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json'
df = load_pc_data(pc_data_file, ['class','level','hit points mean','hit points multiplier'])
df = df[df['class'].isin(classes)]
df['adj hit points mean'] = df['hit points mean']*df['hit points multiplier']
for c in classes:
    dfG = df[df['class'].eq(c)][['level','adj hit points mean']].groupby(['level']).median().reset_index()
    fig.add_trace(go.Scatter(
        x=dfG['level'].to_numpy(), 
        y=dfG['adj hit points mean'].to_numpy(),
        mode='lines+markers', 
        line=dict(color=PC_CLASSES[c]['color'], width=1),
        marker=dict(color=PC_CLASSES[c]['color']),
        unselected=dict(marker=dict(opacity=0.0)),
        name=c,
        hovertemplate=
                f'<b>{c}</b><br>'+
                'level %{x}<br>'+
                'hit points %{y:.0f}'+
                '<extra></extra>'
    ))


# average
"""
dfG = df[['level','adj hit points mean']].groupby(['level']).mean().reset_index()
fig.add_trace(go.Scatter(
    x=dfG['level'].to_numpy(), 
    y=dfG['adj hit points mean'].to_numpy(),
    mode='lines+markers', 
    line=dict(color='black', dash='dash', width=3),
    unselected=dict(marker=dict(opacity=0.0)),
    name='average',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'hit points %{y:.0f}'+
            '<extra></extra>'
))"""

dfG = df[['level','adj hit points mean']].groupby(['level']).median().reset_index()
plot_data_and_fit(
    fig, 
    dfG['level'].to_numpy(), dfG['adj hit points mean'].to_numpy(), 
    name='average',
    legendgroup='average',
    line=dict(color='black', dash='dash'),
    hovertemplate=
            f'<b>Average</b><br>'+
            'level %{x:.0f}<br>'+
            'hit points %{y:.0f}'+
            '<extra></extra>',
    print_coefficients=True,
)

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-adj-hp-vs-level-large', style='aspect-ratio:600/550')
    tfb.save_fig_html(fig, format='small', name=f'./fig-adj-hp-vs-level-small', style='aspect-ratio:600/600')

LVL >= 1: 0.45 + 6.96*LVL


In [34]:
# Plots adjusted hit points for each class, calculated for a full adventuring day with Medium encounters and two short rests.

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='rounds to win',
            range=[0, 9],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='top', y=0.99,
            xanchor='left',   x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# player character data
levels = list(range(1,21))
ac_matrix = []
sb_matrix = []
ab_matrix = []
dc_matrix = []
exclude_pcs = ['Barbarian','Fighter','Rogue']
for c in pcs:
    pc = pcs[c]
    ac_matrix.append([pc.set_level(lvl).build().get_armor_class() for lvl in levels])
    sb_matrix.append([np.mean(list(pc.set_level(lvl).build().get_save_bonuses().values())) for lvl in levels])
    ab_matrix.append([pc.set_level(lvl).build().get_attack_bonus() for lvl in levels])
    if c not in exclude_pcs:
        dc_matrix.append([pc.set_level(lvl).build().get_save_dc() for lvl in levels])
p_ac = np.mean(np.array(ac_matrix), axis=0)
p_sb = np.mean(np.array(sb_matrix), axis=0)
p_ab = np.mean(np.array(ab_matrix), axis=0)
p_dc = np.mean(np.array(dc_matrix), axis=0)

"""
level = np.array(range(1,21))
p_ac = np.round(14.7 + level/6, 2)
p_sb = np.round( 1.5 + level/5, 2)
p_ab = np.round( 4.6 + level/3, 2)
p_dc = np.round(12.6 + level/3, 2)
"""

classes = list(PC_CLASSES.keys())
pc_data_file = f'../../_classes/xp-and-player-characters/pc data - Medium 2 short rests.json'
df = load_pc_data(pc_data_file, ['class','level','hit points mean','hit points multiplier','damage per round mean'])
df = df[df['class'].isin(classes)]
df['adj hit points mean'] = df['hit points mean']*df['hit points multiplier']
dfG = df[['level', 'adj hit points mean', 'damage per round mean']].groupby(['level']).mean().reset_index()
p_hp = dfG['adj hit points mean'].to_numpy()
p_dpr = dfG['damage per round mean'].to_numpy()

# monster stats
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['unadj AC'] = dfM.apply(lambda row: unadjusted_armor_class(row), axis=1)
dfM['adj SB'] = dfM.apply(lambda row: average_save_bonus(row), axis=1)
dfM['adj SB'] = dfM['adj SB'] + dfM.apply(lambda row: 4 if 'magic resistance' in row['AC adjustments'] else 0, axis=1)
dfM['adj DC'] = dfM['adj AB'] + 8
dfM = dfM[['CR','adj HP','AC','adj AC','adj SB','adj DPR','adj AB','adj DC','unadj AC']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 20)]
dfM.sort_values(by='CR', inplace=True)

m_ac  = dfM['unadj AC'].to_numpy()
m_sb  = dfM['adj SB'].to_numpy()
m_ab  = dfM['adj AB'].to_numpy()
m_dc  = dfM['adj DC'].to_numpy()
m_dpr = dfM['adj DPR'].to_numpy()
m_hp  = dfM['adj HP'].to_numpy()

"""
level = np.array(range(1,21))
m_ac  = np.round(13.0 + level/3, 2)
m_sb  = np.round( 0.0 + level/2, 2)
m_ab  = np.round( 3.5 + level/2, 2)
m_dc  = np.round(11.5 + level/2, 2)
m_dpr =  6 +  6*level
m_hp  = 16 + 16*level
"""


# monsters
hit_prob  = np.array([attack_hit_crit_prob(ac, ab) for ac, ab in zip(p_ac, m_ab)])
fail_prob = np.array([save_fail_prob(sb, dc) for sb, dc in zip(p_sb, m_dc)])
rounds_mean = 4*p_hp/(0.5*(hit_prob + fail_prob)*m_dpr)
plot_data_and_fit(
    fig, 
    levels, rounds_mean,
    name='monsters',
    legendgroup='monsters',
    line=dict(color=COLOR_LIST[0], dash='solid', width=1),
    hovertemplate=
            '<b>Monsters</b><br>'+
            'level %{x}<br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>',
    print_coefficients=True,
)
print('monsters:', np.mean(rounds_mean))


# player characters
p_hit = np.array([attack_hit_crit_prob(ac, ab) for ac, ab in zip(m_ac, p_ab)])
p_fail = np.array([save_fail_prob(sb, dc) for sb, dc in zip(m_sb, p_dc)])
rounds_mean = m_hp/(4*0.5*(p_hit + p_fail)*p_dpr)
plot_data_and_fit(
    fig, 
    levels, rounds_mean,
    name='player characters',
    legendgroup='player characters',
    line=dict(color=COLOR_LIST[1], dash='solid', width=1),
    hovertemplate=
            '<b>Player Characters</b><br>'+
            'level %{x}<br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>',
    print_coefficients=True,
)
print('player characters:', np.mean(rounds_mean))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-rounds-to-win-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-rounds-to-win-small')

LVL >= 1: 6.66 + 0.02*LVL
monsters: 6.832871995143565
LVL >= 1: 2.06 + 0.06*LVL
player characters: 2.673155372137044
